# Noisy 1B4 (Basemodel Training)
This model is a custom 1.4B model containing
- 96 layers
- 1024 embedding size

It was initialized using the original RWKV trainer here : https://github.com/PicoCreator/RWKV-LM-LoRA/blob/picocreator-init-memory-experiment/notebook/echo-B-1B4-training.ipynb

The goal of this experiment, is to measure two seperate objective
- quantify and observe the training of a deep layered model
- test for overfitting conditions across multiple epoch
- test if overfitting is mitigated with the introduction of a noisy state

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

# Basic Setup

In [ ]:
# # First lets get the blank init model, these init model was generated
# # using the original RWKV-LM repo (as at this point of writing, this repo cannot init a model)
# #
# # As such I have preinitialized these blank models and uploaded them to HF for convinence
# #
# # You can also download the model for each stage respectively
# #
# # Comment back in this cell if you need it
# !mkdir -p ../../../model/
# !mkdir -p ../../../datapath/
# !mkdir -p ../../../checkpoint/
# !rm -rf ../../../model/Echo-B-1B4-Init.pth
# !cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-B-1B4-Init.pth
# !ls -alh ../../../model/Echo-B-1B4-Init.pth

In [32]:
DEEPSPEED_STRAT="deepspeed_stage_2_offload"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="MultiEpoch-1B4"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_2_offload
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/picocreator/rwkv-proj/picocreator-noise-experiment/notebook/experiment/noisy-training
TRAINER_DIR: /home/picocreator/rwkv-proj/picocreator-noise-experiment/RWKV-v4neo
PROJECT_DIR: /home/picocreator/rwkv-proj/picocreator-noise-experiment


## Stage 1 : Baseline model training

In [6]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_dataset.py "{NOTEBOOK_DIR}/MultiEpoch-1B4-baseline-enwiki.yaml"

Found cached dataset parquet (/home/picocreator/.cache/huggingface/datasets/teven___parquet/teven--enwiki_10k-de63a925546e70ab/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 48.03it/s]


In [10]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B4-baseline-enwiki.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Baseline Enwiki_10k Foundation (ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

[2023-07-17 22:30:19,467] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in ./wandb/run-20230717_223021-o6sshbuo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run MultiEpoch-1B4 - Baseline Enwiki_10k Foundation (ctx=4096, deepspeed_stage_2_offload)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Noise-Training
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Noise-Training/runs/o6sshbuo
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_4096_bf16/build.ninja...
Building extension modul

In [11]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B4-Baseline/last.ckpt" "../model/MultiEpoch-1B4-Baseline.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B4-Baseline.pth"

[2023-07-18 01:48:08,600] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/MultiEpoch-1B4-Baseline/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.gradients, world_size: 1
Parsing checkpoint created by deepspeed==0.9.5
Reconstructed fp32 state dict with 1734 params 1412675584 elements
Saving fp32 state dict to ../model/MultiEpoch-1B4-Baseline.pth
-rw-rw-r-- 1 picocreator picocreator 5.3G Jul 18 01:49 ../model/MultiEpoch-1B4-Baseline.pth


In [12]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py ../model/MultiEpoch-1B4-Baseline.pth "cuda fp32"

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading ../model/MultiEpoch-1B4-Baseline.pth ...
Strategy: (total 96+1=97 layers)
* cuda [float32, float32], store 97 layers
0-cuda-float32-float32 1-cuda-float32-float32 2-cuda-float32-float32 3-cuda-float32-float32 4-cuda-float32-float32 5-cuda-float32-float32 6-cuda-float32-float32 7-cuda-float32-float32 8-cuda-float32-float32 9-cuda-float32-float32 10-cuda-float32-float32 11-cuda-float32-float32 12-cuda-float32-float32 13-cuda-float32-float32 14-cuda-float32-float32 15-cuda-float32-float32 16-cuda-float32-float32 17-cuda-float32-float32 18-cuda-float32-float32 19-cuda-float32-float32 20-cuda-float32-float32 21-cuda-float32-float32 22-cuda-float32-float32 23-cuda-float32-float32 24-cuda-float32-float32 25-cuda-float32-float32 26-cuda-float32-float32 27-cuda-float32-float32 28-cuda-float32-float32 29-cuda-float32-float32 30-cuda-float32-float32 31-cuda-float32-float32 32-cuda-float32-float32 33-cuda-float32-float32 34-cuda-float32-float32

## Stage 2 : Noisy model training

In [34]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python new_train.py fit \
        -c "{NOTEBOOK_DIR}/MultiEpoch-1B4-noisy-enwiki.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Noisy Enwiki_10k Foundation (ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" 

[2023-07-18 14:38:31,486] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1'
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.15.5
wandb: Run data is saved locally in ./wandb/run-20230718_143834-qjwbc0lg
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run MultiEpoch-1B4 - Noisy Enwiki_10k Foundation (ctx=4096, deepspeed_stage_2_offload)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Noise-Training
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Noise-Training/runs/qjwbc0lg
Using /home/picocreator/.cache/torch_extensions/py311_cu117 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/picocreator/.cache/torch_extensions/py311_cu117/wkv_4096_bf16/build.ninja...
Building extension module w

In [ ]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/MultiEpoch-1B4-Noisy/last.ckpt" "../model/MultiEpoch-1B4-Noisy.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/MultiEpoch-1B4-Noisy.pth"

[2023-07-18 14:38:15,998] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/MultiEpoch-1B4-Noisy/last.ckpt/checkpoint'
Traceback (most recent call last):
  File "/home/picocreator/rwkv-proj/picocreator-noise-experiment/RWKV-v4neo/export_checkpoint.py", line 623, in <module>
    convert_zero_checkpoint_to_fp32_state_dict(args.checkpoint_dir, output_file)
  File "/home/picocreator/rwkv-proj/picocreator-noise-experiment/RWKV-v4neo/export_checkpoint.py", line 537, in convert_zero_checkpoint_to_fp32_state_dict
    state_dict = get_fp32_state_dict_from_zero_checkpoint(checkpoint_dir, tag)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/picocreator/rwkv-proj/picocreator-noise-experiment/RWKV-v4neo/export_checkpoint.py", line 523, in get_fp32_state_dict_from_zero_checkpoint
    return _get_fp32_state_dict_from_zero_checkpoint(ds_checkpoint_dir)
           ^^^^^^^^^

In [ ]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && python3 dragon_test.py ../model/MultiEpoch-1B4-Noisy.pth "cuda fp32"

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 0

Loading ../model/MultiEpoch-1B4-Noisy.pth ...
Traceback (most recent call last):
  File "/home/picocreator/rwkv-proj/picocreator-noise-experiment/RWKV-v4neo/dragon_test.py", line 70, in <module>
    model = RWKV(model=MODEL_PATH, strategy=MODEL_STRATEGY)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/torch/jit/_script.py", line 292, in init_then_script
    original_init(self, *args, **kwargs)
  File "/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/rwkv/model.py", line 104, in __init__
    self.w = torch.load(args.MODEL_NAME, map_location='cpu') # load model to CPU first
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/picocreator/anaconda3/envs/rwkv-infctx/lib/python3.11/site-packages/torch/serialization.py", line 791, in load
    with _open_file_like(f, 'rb') as opened_file:
         ^^^^^^^^^^^